In [46]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyodbc
import warnings
from matplotlib.lines import Line2D
import scipy.stats
import matplotlib.ticker as mtick
warnings.filterwarnings('ignore')
#pd.set_option("display.max_columns", None)
from IPython.display import HTML

In [51]:
df = pd.read_csv("HPV_Test_Set.csv")

In [67]:
pd.set_option("display.max_columns", None)
df.head(5)

,SampleUId,TestGuid,ReplicateNumber,SampleID,SampleType,SampleSpecimenType,TestSpecimenType,SpecimenTubeType,AssayName,ResultCode,AssayVersion,RpVersion,Channel,TargetName,ProcessingStep,ChannelSummarySetId,LocalizedResult,Ct,EndPointFluorescence,MaxPeakHeight,EPR,LogConc,Conc,ReadingsSetId,Readings1,Readings2,Readings3,Readings4,Readings5,Readings6,Readings7,Readings8,Readings9,Readings10,Readings11,Readings12,Readings13,Readings14,Readings15,Readings16,Readings17,Readings18,Readings19,Readings20,Readings21,Readings22,Readings23,Readings24,Readings25,Readings26,Readings27,Readings28,Readings29,Readings30,Readings31,Readings32,Readings33,Readings34,Readings35,Readings36,Readings37,Readings38,Readings39,Readings40,Readings41,Readings42,Readings43,Readings44,Readings45,Readings46,Readings47,Readings48,Readings49,Readings50,Readings51,Readings52,Readings53,Readings54,Readings55,Readings56,Readings57,Readings58,Readings59,Readings60,Readings61,Readings62,Readings63,Readings64,Readings65,Readings66,Readings67,Readings68,Readings69,Readings70,Readings71,Readings72,Readings73,Readings74,Readings75,Readings76,Readings77,Readings78,Readings79,Readings80,Readings81,Readings82,Readings83,Readings84,Readings85,Readings86,Readings87,Readings88,Readings89,Readings90,Readings91,Readings92,Readings93,Readings94,Readings95,Readings96,Readings97,Readings98,Readings99,Readings100,NeuMoDxSystemUId,NeuMoDxSystemId,N500SerialNumber,NeuMoDxSoftwareVersion,HamiltonSerialNumber,HamiltonFirmwareVersion,HamiltonMethodVersion,HamiltonSoftwareVersion,HeaterModuleUId,HeaterModuleId,HeaterModuleSerial,HeaterFirmware,HeaterModuleIndex,CapturePlateWell,PcrModuleUId,PcrModuleId,PcrModuleSerial,ABFirmwareVersion,PcrFirmwareVersion,PcrFirmwareVersion.1,PCRModuleIndex,PcrCartridgeLane,TestStripNeuMoDxCarrier,TestStripNeuMoDxCarrierPosition,TestStripNeuMoDxWell,TestStripLDTMasterMixCarrier,TestStripLDTMasterMixCarrierPosition,TestStripLDTMasterMixWell,TestStripLDTPrimerProbeCarrier,TestStripLDTPrimerCarrierPosition,TestStripLDTPrimerProbeWell,TestOperator,RunType,Status,SampleOrigin,StartDateTime,LHPAStartDateTime,LHPBStartDateTime,LHPCStartDateTime,PCRStartDateTime,EndDateTime,SampleTubeRack,SampleTubePosition,BulkReagentDrawer,LHPAADPPosition,LHPCADPPosition,BufferLargeTipRackBarcode,BufferLargeTipPosition,SampleLargeTipRackBarcode,SampleLargeTipPosition,SmallTipRackBarcode,FilterLargeTipRackBarcode,FilterLargeTipPosition,CartridgeId,CartridgeLot,CartridgeSerial,CartridgeExpirationDate,BufferTroughId,BufferTroughLot,BufferTroughSerial,BufferTroughExpirationDate,ExtractionPlateId,ExtractionPlateLot,ExtractionPlateSerial,ExtractionPlateExpirationDate,TestStripId,TestStripLot,TestStripSerial,TestStripExpirationDate,ReleaseReagentId,ReleaseReagentLot,ReleaseReagentSerial,ReleaseReagentExpirationDate,WashReagentId,WashReagentLot,WashReagentSerial,WashReagentExpirationDate,LDTTestStripMMId,LDTTestStripMMLot,LDTTestStripMMSerial,LDTTestStripMMExpirationDate,LDTTestStripPPMId,LDTTestStripPPMLot,LDTTestStripPPMSerial,LDTTestStripPPMExpirationDate
0,38580bae-d3d0-47db-b532-003876e62f3a,81898b37-4aaa-ec11-8665-1866da4c1375,1,HPVpos1,Patient Sample,Cytology,Cytology,SecondaryTube13x75,HPV,HPV,4.2.1,2.1.9.5,Green_470_510,HPV 16,Normalized,b9f9732a-0567-4d17-8bc2-61769d897841,TargetAmplified,28.72,12501,1000.52,1.79,0,0,63103ab0-dc2b-4a37-b296-4fdb9696a220,83.84,56.56,39.68,25.39,32.71,21.02,25.74,13.45,-2.83,2.68,-4.60,-18.29,-21.77,-24.06,-29.54,-18.43,-21.91,-22.00,-22.28,-20.57,-14.85,10.06,28.58,86.89,158.21,285.52,542.64,1016.76,1710.07,2709.79,3943.70,5277.62,6653.53,7934.45,9093.96,10101.88,10955.19,11647.51,12176.22,12500.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f5be28f9-aee4-48a2-abe8-dedf8ae0035a,0933ebd4-a32b-4b5f-a045-f02ca549a4fd,N000009,1.9.2.6,B733,3.4S e 2016-0

In [97]:
dfm = df.copy()

In [103]:
dfm = dfm[dfm.ProcessingStep == "Raw"]
dfm = dfm[dfm.Channel == "Yellow_530_555"]
total = dfm.shape[0]
print(total)
#dfm = dfm[dfm.LocalizedResult == "TargetAmplified"]
dfm = dfm[dfm.SampleID.str[:6] != "HPVpos"]
dfm = dfm[dfm.MaxPeakHeight > 0]
dfm = dfm[dfm.Ct < 10]
dfm = dfm.sort_values(by=["MaxPeakHeight"],ascending=False)
dfm.filter(items= ["SampleUId","SampleID","ResultCode",
                    "AssayVersion","Channel","LocalizedResult",
                    "Ct","EndPointFluorescence","MaxPeakHeight","EPR",
                    'N500SerialNumber','NeuMoDxSoftwareVersion','PcrModuleSerial'])

print("{0:.2%}".format(dfm.shape[0]/total))



dfm = df.copy()

497
0.80%


In [80]:
dfline

,SampleUId,TestGuid,ReplicateNumber,SampleID,SampleType,SampleSpecimenType,TestSpecimenType,SpecimenTubeType,AssayName,ResultCode,AssayVersion,RpVersion,Channel,TargetName,ProcessingStep,ChannelSummarySetId,LocalizedResult,Ct,EndPointFluorescence,MaxPeakHeight,EPR,LogConc,Conc,ReadingsSetId,Readings1,Readings2,Readings3,Readings4,Readings5,Readings6,Readings7,Readings8,Readings9,Readings10,Readings11,Readings12,Readings13,Readings14,Readings15,Readings16,Readings17,Readings18,Readings19,Readings20,Readings21,Readings22,Readings23,Readings24,Readings25,Readings26,Readings27,Readings28,Readings29,Readings30,Readings31,Readings32,Readings33,Readings34,Readings35,Readings36,Readings37,Readings38,Readings39,Readings40,Readings41,Readings42,Readings43,Readings44,Readings45,Readings46,Readings47,Readings48,Readings49,Readings50,Readings51,Readings52,Readings53,Readings54,Readings55,Readings56,Readings57,Readings58,Readings59,Readings60,Readings61,Readings62,Readings63,Readings64,Readings65,Readings66,Readings67,Readings68,Readings69,Readings70,Readings71,Readings72,Readings73,Readings74,Readings75,Readings76,Readings77,Readings78,Readings79,Readings80,Readings81,Readings82,Readings83,Readings84,Readings85,Readings86,Readings87,Readings88,Readings89,Readings90,Readings91,Readings92,Readings93,Readings94,Readings95,Readings96,Readings97,Readings98,Readings99,Readings100,NeuMoDxSystemUId,NeuMoDxSystemId,N500SerialNumber,NeuMoDxSoftwareVersion,HamiltonSerialNumber,HamiltonFirmwareVersion,HamiltonMethodVersion,HamiltonSoftwareVersion,HeaterModuleUId,HeaterModuleId,HeaterModuleSerial,HeaterFirmware,HeaterModuleIndex,CapturePlateWell,PcrModuleUId,PcrModuleId,PcrModuleSerial,ABFirmwareVersion,PcrFirmwareVersion,PcrFirmwareVersion.1,PCRModuleIndex,PcrCartridgeLane,TestStripNeuMoDxCarrier,TestStripNeuMoDxCarrierPosition,TestStripNeuMoDxWell,TestStripLDTMasterMixCarrier,TestStripLDTMasterMixCarrierPosition,TestStripLDTMasterMixWell,TestStripLDTPrimerProbeCarrier,TestStripLDTPrimerCarrierPosition,TestStripLDTPrimerProbeWell,TestOperator,RunType,Status,SampleOrigin,StartDateTime,LHPAStartDateTime,LHPBStartDateTime,LHPCStartDateTime,PCRStartDateTime,EndDateTime,SampleTubeRack,SampleTubePosition,BulkReagentDrawer,LHPAADPPosition,LHPCADPPosition,BufferLargeTipRackBarcode,BufferLargeTipPosition,SampleLargeTipRackBarcode,SampleLargeTipPosition,SmallTipRackBarcode,FilterLargeTipRackBarcode,FilterLargeTipPosition,CartridgeId,CartridgeLot,CartridgeSerial,CartridgeExpirationDate,BufferTroughId,BufferTroughLot,BufferTroughSerial,BufferTroughExpirationDate,ExtractionPlateId,ExtractionPlateLot,ExtractionPlateSerial,ExtractionPlateExpirationDate,TestStripId,TestStripLot,TestStripSerial,TestStripExpirationDate,ReleaseReagentId,ReleaseReagentLot,ReleaseReagentSerial,ReleaseReagentExpirationDate,WashReagentId,WashReagentLot,WashReagentSerial,WashReagentExpirationDate,LDTTestStripMMId,LDTTestStripMMLot,LDTTestStripMMSerial,LDTTestStripMMExpirationDate,LDTTestStripPPMId,LDTTestStripPPMLot,LDTTestStripPPMSerial,LDTTestStripPPMExpirationDate
1,38580bae-d3d0-47db-b532-003876e62f3a,81898b37-4aaa-ec11-8665-1866da4c1375,1,HPVpos1,Patient Sample,Cytology,Cytology,SecondaryTube13x75,HPV,HPV,4.2.1,2.1.9.5,Green_470_510,HPV 16,Raw,b9f9732a-0567-4d17-8bc2-61769d897841,TargetAmplified,28.72,12501,1000.52,1.79,0,0,cab5f4e8-b107-4925-b594-2d4c840eea0a,16503.0,16324.0,16420.0,16387.0,16439.0,16452.0,16381.0,16382.0,16431.0,16398.0,16391.0,16429.0,16366.0,16383.0,16401.0,16400.0,16422.0,16442.0,16386.0,16421.0,16419.0,16451.0,16491.0,16531.0,16534.0,16731.0,16828.0,17148.0,17837.0,18925.0,20218.0,21847.0,23338.0,24527.0,25825.0,26643.0,27665.0,28398.0,28814.0,29307.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f5be28f9-aee4-48a2-abe8-dedf8ae0035a,0933ebd4-a32b-4b5f-a045-f02ca549a4fd,N000

In [ ]:
dfg = dfline[dfline.Channel == "Green_470_510"]
dfy = dfline[dfline.Channel == "Yellow_530_555"]
dfo = dfline[dfline.Channel == "Orange_585_610"]
dffr = dfline[dfline.Channel == "Far_Red_680_715"]
dfr = dfline[dfline.Channel == "Red_625_660"]

,SampleUId,TestGuid,ReplicateNumber,SampleID,SampleType,SampleSpecimenType,TestSpecimenType,SpecimenTubeType,AssayName,ResultCode,AssayVersion,RpVersion,Channel,TargetName,ProcessingStep,ChannelSummarySetId,LocalizedResult,Ct,EndPointFluorescence,MaxPeakHeight,EPR,LogConc,Conc,ReadingsSetId,Readings1,Readings2,Readings3,Readings4,Readings5,Readings6,Readings7,Readings8,Readings9,Readings10,Readings11,Readings12,Readings13,Readings14,Readings15,Readings16,Readings17,Readings18,Readings19,Readings20,Readings21,Readings22,Readings23,Readings24,Readings25,Readings26,Readings27,Readings28,Readings29,Readings30,Readings31,Readings32,Readings33,Readings34,Readings35,Readings36,Readings37,Readings38,Readings39,Readings40,Readings41,Readings42,Readings43,Readings44,Readings45,Readings46,Readings47,Readings48,Readings49,Readings50,Readings51,Readings52,Readings53,Readings54,Readings55,Readings56,Readings57,Readings58,Readings59,Readings60,Readings61,Readings62,Readings63,Readings64,Readings65,Readings66,Readings67,Readings68,Readings69,Readings70,Readings71,Readings72,Readings73,Readings74,Readings75,Readings76,Readings77,Readings78,Readings79,Readings80,Readings81,Readings82,Readings83,Readings84,Readings85,Readings86,Readings87,Readings88,Readings89,Readings90,Readings91,Readings92,Readings93,Readings94,Readings95,Readings96,Readings97,Readings98,Readings99,Readings100,NeuMoDxSystemUId,NeuMoDxSystemId,N500SerialNumber,NeuMoDxSoftwareVersion,HamiltonSerialNumber,HamiltonFirmwareVersion,HamiltonMethodVersion,HamiltonSoftwareVersion,HeaterModuleUId,HeaterModuleId,HeaterModuleSerial,HeaterFirmware,HeaterModuleIndex,CapturePlateWell,PcrModuleUId,PcrModuleId,PcrModuleSerial,ABFirmwareVersion,PcrFirmwareVersion,PcrFirmwareVersion.1,PCRModuleIndex,PcrCartridgeLane,TestStripNeuMoDxCarrier,TestStripNeuMoDxCarrierPosition,TestStripNeuMoDxWell,TestStripLDTMasterMixCarrier,TestStripLDTMasterMixCarrierPosition,TestStripLDTMasterMixWell,TestStripLDTPrimerProbeCarrier,TestStripLDTPrimerCarrierPosition,TestStripLDTPrimerProbeWell,TestOperator,RunType,Status,SampleOrigin,StartDateTime,LHPAStartDateTime,LHPBStartDateTime,LHPCStartDateTime,PCRStartDateTime,EndDateTime,SampleTubeRack,SampleTubePosition,BulkReagentDrawer,LHPAADPPosition,LHPCADPPosition,BufferLargeTipRackBarcode,BufferLargeTipPosition,SampleLargeTipRackBarcode,SampleLargeTipPosition,SmallTipRackBarcode,FilterLargeTipRackBarcode,FilterLargeTipPosition,CartridgeId,CartridgeLot,CartridgeSerial,CartridgeExpirationDate,BufferTroughId,BufferTroughLot,BufferTroughSerial,BufferTroughExpirationDate,ExtractionPlateId,ExtractionPlateLot,ExtractionPlateSerial,ExtractionPlateExpirationDate,TestStripId,TestStripLot,TestStripSerial,TestStripExpirationDate,ReleaseReagentId,ReleaseReagentLot,ReleaseReagentSerial,ReleaseReagentExpirationDate,WashReagentId,WashReagentLot,WashReagentSerial,WashReagentExpirationDate,LDTTestStripMMId,LDTTestStripMMLot,LDTTestStripMMSerial,LDTTestStripMMExpirationDate,LDTTestStripPPMId,LDTTestStripPPMLot,LDTTestStripPPMSerial,LDTTestStripPPMExpirationDate
17,3d61cdac-0e94-47fe-b315-008159283a62,ae796ffe-bcb1-ec11-85aa-5cf370a30ab5,1,G14539,Patient Sample,TransportMedium,TransportMedium,SecondaryTube13x75,FLU-RSV-COV-2,FVN1,4.0.2,2.1.9.5,Green_470_510,FluB,Raw,971ec9f7-9895-43fc-bec7-ce5db64c2ea3,TargetAmplified,32.79,9386.0,627.95,2.59,NaN,NaN,ae6ff537-1fb7-4294-913e-b71e030ed003,5946.0,5962.0,5939.0,5953.0,5949.0,5937.0,5937.0,5932.0,5936.0,5937.0,5940.0,5953.0,5935.0,5954.0,5928.0,5942.0,5933.0,5935.0,5923.0,5940.0,5929.0,5927.0,5945.0,5930.0,5935.0,5929.0,5940.0,5947.0,5992.0,6007.0,6076.0,6261.0,6602.0,7186.0,8045.0,9000.0,9872.0,10740.0,11469.0,12137.0,12686.0,13238.0,13664.0,13998.0,14338.0,14608.0,14857.0,15123.0,15260.0,15416.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45d80334-e23c-4192-884b-33c4ba9df375,70233358-f7